<a href="https://colab.research.google.com/github/SaraVaseem/COMP440/blob/main/Copy_of_brain_tumor_segmentation_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install torch torchvision
!pip install numpy matplotlib opencv-python
!pip install pycocotools

In [36]:
!pip install distinctipy jupyter matplotlib pandas pillow torchtnt==0.2.0 tqdm tabulate
# Install additional utility packages
!pip install cjm_pandas_utils cjm_pil_utils cjm_psl_utils cjm_pytorch_utils cjm_torchvision_tfms

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 99.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 385.8/385.8 kB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.5/133.5 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 62.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 6.6 MB/s eta 0:00:00
  Attempting uninstall: jupyter-client
    Found existing installation: jupyter-client 6.1.12
    Uninstalling jupyter-client-6.1.12:
      Successfully uninstalled jupyter-client-6.1.12
  Attempting uninstall: jupyter-server
    Found existing installation: jupyter-server 1.16.0
    Uninstalling jupyter-server-1.16.0:
      Successfully uninstalled jupyter-server-1

In [37]:
# Import Python Standard Library dependencies
import datetime
from functools import partial
from glob import glob
import json
import math
import multiprocessing
import os
from pathlib import Path
import random
from typing import Any, Dict, Optional

# Import utility functions
from cjm_psl_utils.core import download_file, file_extract, get_source_code
from cjm_pil_utils.core import resize_img, get_img_files, stack_imgs
from cjm_pytorch_utils.core import pil_to_tensor, tensor_to_pil, get_torch_device, set_seed, denorm_img_tensor, move_data_to_device
from cjm_pandas_utils.core import markdown_to_pandas, convert_to_numeric, convert_to_string
from cjm_torchvision_tfms.core import ResizeMax, PadSquare, CustomRandomIoUCrop

# Import the distinctipy module
from distinctipy import distinctipy

# Import matplotlib for creating plots
import matplotlib.pyplot as plt

# Import numpy
import numpy as np

# Import the pandas package
import pandas as pd

# Set options for Pandas DataFrame display
pd.set_option('max_colwidth', None)  # Do not truncate the contents of cells in the DataFrame
pd.set_option('display.max_rows', None)  # Display all rows in the DataFrame
pd.set_option('display.max_columns', None)  # Display all columns in the DataFrame

# Import PIL for image manipulation
from PIL import Image, ImageDraw

# Import PyTorch dependencies
import torch
from torch.amp import autocast
from torch.cuda.amp import GradScaler
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchtnt.utils import get_module_summary
import torchvision
torchvision.disable_beta_transforms_warning()
from torchvision.tv_tensors import BoundingBoxes, Mask
from torchvision.utils import draw_bounding_boxes, draw_segmentation_masks
import torchvision.transforms.v2  as transforms
from torchvision.transforms.v2 import functional as TF

# Import Mask R-CNN
from torchvision.models.detection import maskrcnn_resnet50_fpn_v2, MaskRCNN
from torchvision.models.detection import MaskRCNN_ResNet50_FPN_V2_Weights
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor

# Import tqdm for progress bar
from tqdm.auto import tqdm

In [38]:
# Set the seed for generating random numbers in PyTorch, NumPy, and Python's random module.
seed = 1234
set_seed(seed)

In [39]:
device = get_torch_device()
dtype = torch.float32
device, dtype

('cuda', torch.float32)

In [40]:
import os
import torch
import torchvision
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import numpy as np
from torchvision import transforms
from tqdm import tqdm
import json

In [44]:
from PIL import ImageDraw

class BrainTumorDataset(Dataset):
    def __init__(self, img_dir, mask_dir, transforms=None):
        self.img_dir = img_dir
        self.mask_dir = mask_dir
        self.transforms = transforms
        self.img_files = sorted([f for f in os.listdir(img_dir) if f.endswith('.png')])

    def __len__(self):
        return len(self.img_files)

    def __getitem__(self, idx):
        # Load image
        img_name = self.img_files[idx]
        img_path = os.path.join(self.img_dir, img_name)
        img = Image.open(img_path).convert("RGB")

        # Load corresponding annotation (mask) from json file
        mask_name = img_name.replace('.png', '.json')
        mask_path = os.path.join(self.mask_dir, mask_name)
        with open(mask_path) as f:
            annotations = json.load(f)

        # Convert annotations to mask format
        masks = []
        boxes = []
        for shape in annotations['shapes']:
            points = shape['points']
            # Ensure that points are in the correct format: list of tuples (x, y)
            flat_points = [(point[0], point[1]) for point in points]

            # Convert to polygon (mask)
            mask = Image.new('L', (annotations['imageWidth'], annotations['imageHeight']), 0)
            ImageDraw.Draw(mask).polygon(flat_points, outline=1, fill=1)
            masks.append(np.array(mask))

            # Convert points to bounding box
            min_x = min(flat_points, key=lambda x: x[0])[0]
            max_x = max(flat_points, key=lambda x: x[0])[0]
            min_y = min(flat_points, key=lambda x: x[1])[1]
            max_y = max(flat_points, key=lambda x: x[1])[1]
            boxes.append([min_x, min_y, max_x, max_y])

        # Stack the masks (multi-class)
        masks = np.stack(masks, axis=-1)

        # Convert everything into tensor
        target = {}
        target['boxes'] = torch.tensor(boxes, dtype=torch.float32)
        target['labels'] = torch.ones(len(boxes), dtype=torch.int64)  # Assuming only one class (tumor)
        target['masks'] = torch.tensor(masks, dtype=torch.uint8)

        if self.transforms:
            img = self.transforms(img)

        return img, target


In [45]:
# Define transformations (if needed)
transform = transforms.Compose([transforms.ToTensor()])

# Setup dataset
img_dir = "/content/drive/MyDrive/Imgs"
mask_dir = "/content/drive/MyDrive/Segmentations 501-750"

dataset = BrainTumorDataset(img_dir=img_dir, mask_dir=mask_dir, transforms=transform)

# DataLoader for training
data_loader = DataLoader(dataset, batch_size=2, shuffle=True, collate_fn=lambda x: tuple(zip(*x)))

# Example to inspect first image and target
images, targets = next(iter(data_loader))
print(images[0].shape, targets[0])


torch.Size([3, 512, 512]) {'boxes': tensor([[300.5000, 146.5000, 392.5000, 243.5000]]), 'labels': tensor([1]), 'masks': tensor([[[0],
         [0],
         [0],
         ...,
         [0],
         [0],
         [0]],

        [[0],
         [0],
         [0],
         ...,
         [0],
         [0],
         [0]],

        [[0],
         [0],
         [0],
         ...,
         [0],
         [0],
         [0]],

        ...,

        [[0],
         [0],
         [0],
         ...,
         [0],
         [0],
         [0]],

        [[0],
         [0],
         [0],
         ...,
         [0],
         [0],
         [0]],

        [[0],
         [0],
         [0],
         ...,
         [0],
         [0],
         [0]]], dtype=torch.uint8)}


In [67]:
import torchvision

# Load a pre-trained Mask R-CNN model
model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)

# Modify the model to work with only one class (tumor)
in_features = model.roi_heads.box_predictor.cls_score.in_features

# Update the box predictor to classify 2 classes (background and tumor)
model.roi_heads.box_predictor = torchvision.models.detection.faster_rcnn.FastRCNNPredictor(in_features, 2)  # 2 classes: background and tumor

# Modify the mask predictor for the tumor class
model.roi_heads.mask_predictor = torchvision.models.detection.mask_rcnn.MaskRCNNPredictor(in_features, 256, 2)  # 2 classes


In [68]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [69]:
model.to(device)

MaskRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(in

In [70]:
images = [img.to(device) for img in images]
targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

In [71]:
print(images[0].shape)  # Check shape of a single image

torch.Size([3, 512, 512])


In [72]:
# Training loop
num_epochs = 10
optimizer = torch.optim.SGD(model.parameters(), lr=0.005, momentum=0.9, weight_decay=0.0005)

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, targets in tqdm(data_loader, desc=f"Epoch {epoch + 1}/{num_epochs}"):
        images = [img.to(device) for img in images]
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        optimizer.zero_grad()
        loss_dict = model(images, targets)

        losses = sum(loss for loss in loss_dict.values())
        losses.backward()
        optimizer.step()
        running_loss += losses.item()

    print(f"Epoch {epoch + 1} Loss: {running_loss / len(data_loader)}")

# Save the model
torch.save(model.state_dict(), "mask_rcnn_brain_tumor.pth")


Epoch 1/10:   0%|          | 0/125 [00:00<?, ?it/s]


RuntimeError: Given transposed=1, weight of size [1024, 256, 2, 2], expected input[31, 256, 14, 14] to have 1024 channels, but got 256 channels instead

In [ ]:
model.eval()
with torch.no_grad():
    for images, targets in tqdm(data_loader, desc="Evaluating"):
        images = [img.to(device) for img in images]
        predictions = model(images)

        # Example: print first prediction's boxes and labels
        print(predictions[0]['boxes'])
        print(predictions[0]['labels'])